# Imports

In [1]:
import os
import time
import pickle
from datetime import datetime

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from tqdm.notebook import tqdm

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR

from util_func import sMAPE, RMSE, MAE

# Config enviroment

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Prepare data

In [3]:
def read_epcor(data_root, force_reload=False):
    try:
        with open(data_root+"loaded_dataset.pk", 'rb') as f:
            df = pickle.load(f)
    except:
        force_reload = True
    
    if force_reload:
        df = pd.read_csv(data_root+"cons_data.csv")
        df["DATETIME"] = df.apply(lambda x: datetime.strptime(str(x["DATE"])+str(x["HOUR_ENDING"]-1), "%Y%m%d%H"), axis=1)
        df.drop("DATE", axis=1, inplace=True)
        df = df[["SITE_ID", "RATE_CLASS", "DATETIME", "IS_DAYLIGHT_SAVING", "CONSUMPTION_KWH"]]
        df.sort_values(["SITE_ID", "DATETIME"], ascending=True, inplace=True)

        with open(data_root+"loaded_dataset.pk", 'wb') as f:
            pickle.dump(df, f)

    return df

def preprocess_epcor(df, train_prop, look_back):
    train_size = int(np.ceil(df.shape[0] * train_prop))
    n_static = 0

    df = df.copy()
    df["YEAR"] = df.apply(lambda x: x["DATETIME"].year - 2018, axis=1)
    df["MONTH"] = df.apply(lambda x: x["DATETIME"].month, axis=1)
    df["DAY"] = df.apply(lambda x: x["DATETIME"].day, axis=1)
    df["WEEKDAY"] = df.apply(lambda x: x["DATETIME"].weekday(), axis=1)
    df["HOUR"] = df.apply(lambda x: x["DATETIME"].hour, axis=1)
    df = df[["SITE_ID", "YEAR", "MONTH", "DAY", "WEEKDAY", "HOUR", "IS_DAYLIGHT_SAVING", "CONSUMPTION_KWH", "RATE_CLASS"]]

    # TODO: try (0.1, 1) scale
    sc = MinMaxScaler()
    features = ["MONTH", "DAY", "WEEKDAY", "HOUR", "CONSUMPTION_KWH"]
    df[features] = sc.fit_transform(df[features])
    df["IS_DAYLIGHT_SAVING"] = df["IS_DAYLIGHT_SAVING"].astype(int)
    # one_hot = pd.get_dummies(df["RATE_CLASS"])
    df.drop("RATE_CLASS", axis=1, inplace=True)
    # df = df.join(one_hot)
    # n_static = one_hot.shape[1]

    data = df.to_numpy()
    inputs = []
    labels = []

    for i in tqdm(range(look_back, len(data))):
        if len(np.unique(data[i-look_back:i, 0])) == 1:
            
            inputs.append(data[i-look_back:i,1:])
            labels.append(data[i,-n_static-1])

    inputs = np.array(inputs)
    labels = np.array(labels).reshape(-1,1)

    X_train = inputs[:train_size]
    y_train = labels[:train_size]

    X_test = inputs[train_size:]
    y_test = labels[train_size:]

    return (X_train, y_train), (X_test, y_test), n_static

# Models

In [4]:
class RNN(nn.Module):
    def __init__(self, hidden_dim, output_dim, n_layers, n_static) -> None:
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.n_static = n_static

        self.rnn = None
        self.fc = nn.Linear(hidden_dim, output_dim)
        # self.fc1 = nn.Linear(hidden_dim, hidden_dim//4)
        # self.fc2 = nn.Linear(hidden_dim//4 + n_static, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.rnn(x, h)
        out = self.fc(self.relu(out[:,-1]))
        # out, h = self.rnn(x[:,:,:-self.n_static], h)
        # out = self.fc1(self.relu(out[:,-1]))
        # out = self.fc2(torch.cat((self.relu(out), x[:,-1,-self.n_static:]), 1))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

class GRUNet(RNN):
    '''GRU'''
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, n_static, drop_prob=0.2):
        super(GRUNet, self).__init__(hidden_dim, output_dim, n_layers, n_static)
        self.rnn = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        

class LSTMNet(RNN):
    '''LSTM'''
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, n_static, drop_prob=0.2):
        super(LSTMNet, self).__init__(hidden_dim, output_dim, n_layers, n_static)
        self.rnn = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)

# Train and Evaluate

In [5]:
def train(model, train_loader, optimizer, criterion):
    model_type = model.__doc__ 
    model_device = 'cuda' if next(model.parameters()).is_cuda else 'cpu'
    batch_size = train_loader.batch_size
    model.train()
    
    start_time = time.process_time()
    h = model.init_hidden(batch_size)
    total_loss = 0
    counter = 0
    for X, y in train_loader:
        batch_size
        counter += 1
        if model_type == "GRU":
            h = h.data
        else:
            h = tuple([e.data for e in h])
        model.zero_grad()
        
        out, h = model(X.to(model_device).float(), h)
        loss = criterion(out, y.to(model_device).float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if counter%200 == 0:
            print("Epoch {}......Step: {}/{}....... Average Loss for this step: {}".format(epoch, counter, len(train_loader), total_loss/counter))
    current_time = time.process_time()
    print("Epoch {}/{} Done, Average Loss: {}".format(epoch, epochs, total_loss/len(train_loader)))
    print("Time Elapsed for Epoch: {} seconds".format(str(current_time-start_time)))
        
def evaluate(model, test_loader, criterion):
    model_type = model.__doc__
    num_batches = len(test_loader)
    model_device = 'cuda' if next(model.parameters()).is_cuda else 'cpu'
    
    model.eval()
    test_loss = 0
    h = model.init_hidden(test_loader.batch_size)
    predicted_values, targets = np.empty((0,1)), np.empty((0,1))
    with torch.no_grad():
        for X, y in test_loader:
            if model_type == "GRU":
                h = h.data
            else:
              h = tuple([e.data for e in h])

            out, h = model(X.float().to(model_device), h)
            test_loss += criterion(out, y.to(model_device)).item()
            predicted_values = np.concatenate((predicted_values, out.cpu().detach().numpy().reshape(-1,1)))
            targets = np.concatenate((targets, y.numpy()))
        
    test_loss /= num_batches
    
    print(f"Test results: \n sMAPE: {sMAPE(predicted_values, targets):>0.2f}% \
                          \n RMSE:  {RMSE(predicted_values, targets):>0.2f} \
                          \n MAE:   {MAE(predicted_values, targets):>0.2f} \
                          \n Avg loss: {test_loss:>8f} \n")

# Main

In [17]:
# Constants
data_root = "../Datasets/Time_Series_Datasets/EPCOR/"
train_prop = 0.9
look_back = 12
batch_size = 256
hidden_dim = 256
input_dim = 7
output_dim = 1
n_layers = 2
epochs = 20
lr = 0.001

# Read Dataset
df = read_epcor(data_root)
df_100 = df[(df["SITE_ID"] < 10) & (df["RATE_CLASS"] == "Residential")]
(X_train, y_train), (X_test, y_test), n_static = preprocess_epcor(df_100, train_prop, look_back)

# Create Datasets
train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

# Create Dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("==> Use accelerator: ", device)

  0%|          | 0/299254 [00:00<?, ?it/s]

==> Use accelerator:  cuda


In [18]:
# model = torch.load("../trained_models/epcor/trained_gru.model")
model = GRUNet(input_dim, hidden_dim, output_dim, n_layers, n_static)

In [ ]:
# model = torch.load("../trained_models/epcor/trained_lstm.model")
model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers, n_static)

In [19]:
model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr)
scheduler = ExponentialLR(optimizer, gamma=0.9)

print("==> Start training ...")
print("Training of {} model".format(model.__doc__))

for epoch in range(1,epochs+1):
    print(f"Epoch {epoch}")
    train(model, train_loader, optimizer, criterion)
    evaluate(model, test_loader, criterion)

    # Update learning rate
    if epoch % 5 == 0:
        scheduler.step() 

print("Task done!")

==> Start training ...
Training of GRU model
Epoch 1
Epoch 1......Step: 200/1052....... Average Loss for this step: 0.00023898845031453675
Epoch 1......Step: 400/1052....... Average Loss for this step: 0.0001386968839238989
Epoch 1......Step: 600/1052....... Average Loss for this step: 0.00010835570379564767
Epoch 1......Step: 800/1052....... Average Loss for this step: 8.87626659289964e-05
Epoch 1......Step: 1000/1052....... Average Loss for this step: 7.231415008652676e-05
Epoch 1/20 Done, Average Loss: 7.005318364147825e-05
Time Elapsed for Epoch: 6.634225850000007 seconds
Test results: 
 sMAPE: 65.80%                           
 RMSE:  0.00                           
 MAE:   0.00                           
 Avg loss: 0.000000 

Epoch 2
Epoch 2......Step: 200/1052....... Average Loss for this step: 2.977979165734723e-05
Epoch 2......Step: 400/1052....... Average Loss for this step: 1.738652946691843e-05
Epoch 2......Step: 600/1052....... Average Loss for this step: 1.528773735576768

In [ ]:
torch.save(model, '../trained_models/epcor/trained_gru.model')
# torch.save(model, '../trained_models/epcor/trained_lstm.model')